# data_world.csv

Tylko energia grupowana po regionie i czasie, energia dyskretna. Tylko transformer.

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import warnings
warnings.filterwarnings('ignore')

2023-10-02 18:00:37.746028: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-02 18:00:40.223957: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
class PositionalEmbedding(tf.keras.layers.Layer):
    def __init__(self, vocab_size, d_model):
        super().__init__()
        self.d_model = d_model
        self.embedding = tf.keras.layers.Embedding(vocab_size, d_model, mask_zero=True) 
        self.pos_encoding = tf.keras.layers.Embedding(100, d_model)

    def compute_mask(self, *args, **kwargs):
        return self.embedding.compute_mask(*args, **kwargs)

    def call(self, x):
        length = tf.shape(x)[1]
        x = self.embedding(x)
        x_pos = tf.range(length, dtype=tf.int32)
        x_pos = self.pos_encoding(x_pos)
        x = x + x_pos
        return x

In [3]:
class BaseAttention(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__()
        self.mha = tf.keras.layers.MultiHeadAttention(**kwargs)
        self.layernorm = tf.keras.layers.LayerNormalization()
        self.add = tf.keras.layers.Add()

In [4]:
class CrossAttention(BaseAttention):
    def call(self, x, context):
        attn_output = self.mha(
          query=x,
          key=context,
          value=context)

        x = self.add([x, attn_output])
        x = self.layernorm(x)

        return x

In [5]:
class GlobalSelfAttention(BaseAttention):
    def call(self, x):
        attn_output = self.mha(
            query=x,
            value=x,
            key=x)
        x = self.add([x, attn_output])
        x = self.layernorm(x)
        return x

In [6]:
class CausalSelfAttention(BaseAttention):
    def call(self, x):
        attn_output = self.mha(
            query=x,
            value=x,
            key=x,
            use_causal_mask = True)
        x = self.add([x, attn_output])
        x = self.layernorm(x)
        return x

In [7]:
class FeedForward(tf.keras.layers.Layer):
    def __init__(self, d_model, dff, dropout_rate=0.1):
        super().__init__()
        self.seq = tf.keras.Sequential([
        tf.keras.layers.Dense(dff, activation='relu'),
        tf.keras.layers.Dense(d_model),
        tf.keras.layers.Dropout(dropout_rate)
        ])
        self.add = tf.keras.layers.Add()
        self.layer_norm = tf.keras.layers.LayerNormalization()

    def call(self, x):
        x = self.add([x, self.seq(x)])
        x = self.layer_norm(x) 
        return x

In [8]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self,*, d_model, num_heads, dff, dropout_rate=0.1):
        super().__init__()

        self.self_attention = GlobalSelfAttention(
            num_heads=num_heads,
            key_dim=d_model,
            dropout=dropout_rate)

        self.ffn = FeedForward(d_model, dff)

    def call(self, x):
        x = self.self_attention(x)
        x = self.ffn(x)
        return x

In [41]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, *, num_layers, d_model, num_heads,
                dff, vocab_size, dropout_rate=0.1):
        super().__init__()
        self.d_model = d_model
        self.num_layers = num_layers
        self.emb_x = tf.keras.layers.Embedding(vocab_size, d_model)
        self.emb_y = tf.keras.layers.Embedding(vocab_size, d_model)
        self.conc = tf.keras.layers.Concatenate(axis=-2)

        self.enc_layers = [
            EncoderLayer(d_model=d_model,
                        num_heads=num_heads,
                        dff=dff,
                        dropout_rate=dropout_rate)
            for _ in range(num_layers)]
        self.dropout = tf.keras.layers.Dropout(dropout_rate)

    def call(self, features):
        pos = features
        x = self.emb_x(pos[:,0][:, tf.newaxis])
        y = self.emb_y(pos[:,1][:, tf.newaxis])
        x = self.conc([x,y])
        x = self.dropout(x)
        for layer in self.enc_layers:
            x = layer(x)

        return x  # Shape `(batch_size, seq_len, d_model)`.

In [42]:
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self,
                *,
                d_model,
                num_heads,
                dff,
                dropout_rate=0.1):
        super(DecoderLayer, self).__init__()

        self.causal_self_attention = CausalSelfAttention(
            num_heads=num_heads,
            key_dim=d_model,
            dropout=dropout_rate)

        self.cross_attention = CrossAttention(
            num_heads=num_heads,
            key_dim=d_model,
            dropout=dropout_rate)

        self.ffn = FeedForward(d_model, dff)

    def call(self, x, context):
        x = self.causal_self_attention(x=x)
        x = self.cross_attention(x=x, context=context)

        x = self.ffn(x)  # Shape `(batch_size, seq_len, d_model)`.
        return x

In [43]:
class Decoder(tf.keras.layers.Layer):
    def __init__(self, *, num_layers, d_model, num_heads, dff, vocab_size,
                dropout_rate=0.1):
        super(Decoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.pos_embedding = PositionalEmbedding(vocab_size=vocab_size,
                                                d_model=d_model,)
        self.dropout = tf.keras.layers.Dropout(dropout_rate)
        self.dec_layers = [
            DecoderLayer(d_model=d_model, num_heads=num_heads,
                        dff=dff, dropout_rate=dropout_rate)
            for _ in range(num_layers)]

        self.last_attn_scores = None

    def call(self, x, context):
        # `x` is token-IDs shape (batch, target_seq_len)
        x = self.pos_embedding(x)  # (batch_size, target_seq_len, d_model)

        x = self.dropout(x)

        for i in range(self.num_layers):
            x  = self.dec_layers[i](x, context)

        # The shape of x is (batch_size, target_seq_len, d_model).
        return x

In [44]:
class Transformer(tf.keras.Model):
    def __init__(self, *, num_layers, d_model, num_heads,
                dropout_rate=0.1, vocab_size):
        super().__init__()
        self.encoder = Encoder(num_layers=num_layers, d_model=d_model,
                            num_heads=num_heads, dff=4*d_model,
                            vocab_size=vocab_size,
                            dropout_rate=dropout_rate)

        self.decoder = Decoder(num_layers=num_layers, d_model=d_model,
                            num_heads=num_heads, dff=4*d_model,
                            vocab_size=vocab_size,
                            dropout_rate=dropout_rate)

        self.final_layer = tf.keras.layers.Dense(2)

    def call(self, inputs):
        # To use a Keras model with `.fit` you must pass all your inputs in the
        # first argument.
        x, context  = inputs
        context = self.encoder(context)  # (batch_size, context_len, d_model)
        x = self.decoder(x, context) # (batch_size, target_len, d_model)
        x = x[:, -1, :]
        logits = self.final_layer(x)  # (batch_size, 1, target_vocab_size)

        return logits

In [45]:
df = pd.read_csv('../../data/data_world.csv')
df.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,1930-12-08T08:01:02.000Z,23.261,120.277,15.0,6.3,mw,NaN,NaN,NaN,NaN,...,2015-05-13T18:52:43.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1930-12-03T18:51:47.000Z,18.233,96.298,10.0,7.4,mw,NaN,NaN,NaN,NaN,...,2015-05-13T18:52:43.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1930-12-02T07:01:30.000Z,25.854,98.356,35.0,6.2,mw,NaN,NaN,NaN,NaN,...,2015-05-13T18:52:43.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1930-11-28T07:32:56.000Z,18.779,-106.767,15.0,6.3,mw,NaN,NaN,NaN,NaN,...,2015-05-13T18:52:43.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1930-11-25T19:02:53.000Z,35.050,139.129,15.0,6.9,mw,NaN,NaN,NaN,NaN,...,2015-05-13T18:52:43.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
df = df[["time", "latitude", "longitude", "mag"]]

In [47]:
df["time"] = pd.to_datetime(df["time"], format="%Y-%m-%dT%H:%M:%S.%fZ")
df

,time,latitude,longitude,mag
0,1930-12-08 08:01:02.000,23.2610,120.2770,6.3
1,1930-12-03 18:51:47.000,18.2330,96.2980,7.4
2,1930-12-02 07:01:30.000,25.8540,98.3560,6.2
3,1930-11-28 07:32:56.000,18.7790,-106.7670,6.3
4,1930-11-25 19:02:53.000,35.0500,139.1290,6.9
...,...,...,...,...
797041,2018-09-01 01:14:38.230,-30.4830,-177.9279,4.3
797042,2018-09-01 01:07:59.120,-10.7558,124.3621,4.0
797043,2018-09-01 01:00:13.810,-5.5167,147.1735,4.6
797044,2018-09-01 00:27:11.440,46.8819,155.6566,4.3


In [48]:
time_split = 30
time_cut = dt.datetime(1973, 1, 1)
geo_split = 10

In [49]:
df = df[df["time"] > time_cut]
df["energy"] = 10**(1.44*df["mag"]+5.24)
df["label"] = np.where(df["mag"] > 5, 1, 0)
df["time"] = df["time"].dt.floor('d')
df["time"] = df["time"] - np.min(df["time"])
df["time"] = df["time"].dt.days // time_split
df["time"] = df["time"].astype(int)
df["latitude"] = df["latitude"] // geo_split
df["latitude"] = df["latitude"] + np.abs(np.min(df["latitude"]))
df["longitude"] = df["longitude"] // geo_split
df["longitude"] = df["longitude"] + np.abs(np.min(df["longitude"]))

In [50]:
df_agg = df.groupby(["latitude", "longitude", "time"]).agg({"energy": "sum", "label": "sum"}).reset_index()
df_agg

,latitude,longitude,time,energy,label
0,0.0,13.0,268,1.018591e+12,0
1,0.0,22.0,482,3.767038e+11,0
2,0.0,31.0,375,5.248075e+11,0
3,0.0,31.0,473,7.311391e+11,0
4,0.0,33.0,280,2.703958e+11,0
...,...,...,...,...,...
71216,17.0,30.0,487,1.018591e+12,0
71217,17.0,30.0,489,3.767038e+11,0
71218,17.0,30.0,509,7.311391e+11,0
71219,17.0,30.0,538,3.767038e+11,0


In [51]:
df_agg["label"] = np.where(df_agg["label"] > 0, 1, 0)

In [52]:
def make_ds(df_agg, block_size, vocab_size):
    dfs_train, dfs_val = [], []
    for i in df_agg["latitude"].unique():
        for j in df_agg["longitude"].unique():
            tmp = df_agg[(df_agg["latitude"] == i) & (df_agg["longitude"] == j)]
            if not tmp.empty:
                start = max(tmp["time"].min() - block_size, 0)
                end = min(tmp["time"].max() + block_size, df_agg["time"].max())
                tmp = tmp.set_index("time").reindex(range(start, end)).fillna(0).rename_axis('time').reset_index()
                tmp["label"] = tmp["label"].shift(-1)
                tmp["latitude"] = i
                tmp["longitude"] = j
                n = int(0.8 * len(tmp))
                df_train = tmp[:n]
                df_val = tmp[n:]
                bins_train = np.linspace(df_train["energy"].min(), df_train["energy"].max(), vocab_size)
                bins_val = np.linspace(df_val["energy"].min(), df_val["energy"].max(), vocab_size)
                df_train["energy"] = np.digitize(df_train["energy"], bins_train)
                df_val["energy"] = np.digitize(df_val["energy"], bins_val)
                for idx in range(1, block_size):
                    df_train["energy" + str(idx)] = df_train["energy"].shift(idx)
                    df_val["energy" + str(idx)] = df_val["energy"].shift(idx)
                dfs_train.append(df_train)
                dfs_val.append(df_val)
    df_final_train = pd.concat(dfs_train)
    df_final_val = pd.concat(dfs_val)
    return df_final_train, df_final_val

In [61]:
block_size = 16
vocab_size = 64
df_train, df_val = make_ds(df_agg, block_size, vocab_size)
df_train.dropna(inplace=True)
df_val.dropna(inplace=True)

In [62]:
df_train.head()

,time,latitude,longitude,energy,label,energy1,energy2,energy3,energy4,energy5,energy6,energy7,energy8,energy9,energy10,energy11,energy12,energy13,energy14,energy15
15,267,0.0,13.0,1,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
16,268,0.0,13.0,64,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
17,269,0.0,13.0,1,0.0,64.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
18,270,0.0,13.0,1,0.0,1.0,64.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
19,271,0.0,13.0,1,0.0,1.0,1.0,64.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [63]:
x_train = df_train.drop(["label", "latitude", "longitude", "time"], axis=1).to_numpy()
y_train = df_train["label"].to_numpy()
x_val = df_val.drop(["label", "latitude", "longitude", "time"], axis=1).to_numpy()
y_val = df_val["label"].to_numpy()

In [64]:
x_pos_train = df_train[["longitude", "latitude"]].to_numpy()
x_pos_val = df_val[["longitude", "latitude"]].to_numpy()

In [65]:
# add anoother column whith 1 - label
y_train = y_train.reshape(-1, 1)
y_val = y_val.reshape(-1, 1)
y_train = np.concatenate((y_train, 1 - y_train), axis=1)
y_val = np.concatenate((y_val, 1 - y_val), axis=1)

In [69]:
model = Transformer(num_layers=4, d_model=256, num_heads=4, dropout_rate=0, vocab_size=vocab_size)

In [70]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003),
                loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                metrics=['accuracy', tf.keras.metrics.F1Score()])

In [71]:
model.fit([x_train, x_pos_train], y_train, epochs=40, batch_size=512, validation_data=([x_val, x_pos_val], y_val))

Epoch 1/40
329/329 [==============================] - 264s 661ms/step - loss: 0.2971 - accuracy: 0.9027 - f1_score: 0.6819 - val_loss: 0.2340 - val_accuracy: 0.9077 - val_f1_score: 0.7030
Epoch 2/40
329/329 [==============================] - 257s 783ms/step - loss: 0.2153 - accuracy: 0.9182 - f1_score: 0.7560 - val_loss: 0.2284 - val_accuracy: 0.9118 - val_f1_score: 0.7698
Epoch 3/40
329/329 [==============================] - 251s 765ms/step - loss: 0.2129 - accuracy: 0.9184 - f1_score: 0.7583 - val_loss: 0.2264 - val_accuracy: 0.9121 - val_f1_score: 0.7389
Epoch 4/40
329/329 [==============================] - 252s 768ms/step - loss: 0.2101 - accuracy: 0.9193 - f1_score: 0.7590 - val_loss: 0.2246 - val_accuracy: 0.9118 - val_f1_score: 0.7593
Epoch 5/40
329/329 [==============================] - 255s 772ms/step - loss: 0.2109 - accuracy: 0.9185 - f1_score: 0.7565 - val_loss: 0.2263 - val_accuracy: 0.9112 - val_f1_score: 0.7668
Epoch 6/40
329/329 [==============================] - 259s 7

KeyboardInterrupt: 

In [41]:
model.count_params()

7423490

In [42]:
model.summary()

Model: "transformer_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_2 (Encoder)         multiple                  3175424   
                                                                 
 decoder_2 (Decoder)         multiple                  4247552   
                                                                 
 dense_50 (Dense)            multiple                  514       
                                                                 
Total params: 7423490 (28.32 MB)
Trainable params: 7423490 (28.32 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
